In [ ]:
import pandas as pd
from splinter import Browser
from bs4 import BeautifulSoup
from webdriver_manager.chrome import ChromeDriverManager

In [ ]:
executable_path={'executable_path':ChromeDriverManager().install()}
browser=Browser('chrome',**executable_path, headless=False)

In [ ]:
url='https://chatham.bottleking.com/main.asp?request=search&type=w'
browser.visit(url)

In [ ]:
wine_df=pd.DataFrame(columns=['sku','name','bottlesize','regprice','saleprice',
                              'description','url','country','region','variety','type'])
while True:
    print("Scraping please be patient...")
    html=browser.html
    soup=BeautifulSoup(html,'html.parser')
    wines=soup.find_all('a',{'class':['Srch-producttitle']})
    wines_sku=soup.find_all('span',{'class':['Srch-sku']})
    wines_bottlesize=soup.find_all('span',{'class':['Srch-bottlesize']})
    wines_description=soup.find_all('span',{'class':['Srch-partialDesc']})
    wines_bottleprice=soup.find_all('span',{'class':['RegularPrice']})
    wines_saleprice=soup.find_all('span',{'class':['SalePrice']})
    for i in range(len(wines)):
#        print(wines[i].text,' ',wines_sku[i].text,' ', wines_bottlesize[i].text,' ', wines_bottleprice[i].text,' ', wines_saleprice[i].text)
#        print('                     ',wines_description[i].text)
        sku=wines_sku[i].text.replace('Sku: ','')
        url='https://chatham.bottleking.com/sku'+ sku +'.html'
#        print('Url=',url)
        try:
            browser.visit(url)
            html=browser.html
            soup=BeautifulSoup(html,'html.parser')
            more_info=str(soup.find_all('a',{'class':['reglink']}))
            country=more_info.split('country=')[1]
            country=country.split('">')[0]
            region=more_info.split('region=')[1]
            region=region.split('">')[0]
            variety=more_info.split('variety=')[1]
            variety=variety.split('">')[0]
            wtype=more_info.split('type=W">')[1]
            wtype=wtype.split('<')[0]
#            print(country,' ',region,' ',variety,' ',wtype)
            browser.back()
        except:
            country='N/A'
            region='N/A'
            variety='N/A'
            wtype='N/A'
            url='N/A'
            browser.back()
 
        wine_df=wine_df.append({
            'sku':str(sku),
            'name':str(wines[i].text),
            'bottlesize':str(wines_bottlesize[i].text),
            'regprice':str(wines_bottleprice[i].text),
            'saleprice':str(wines_saleprice[i].text),
            'description':str(wines_description[i].text),
            'url':url,
            'country':country,
            'region':region,
            'variety':variety,
            'type':wtype
        },ignore_index=True)       
#        print('------\n')
    try:
        browser.links.find_by_partial_text('>>').click()
    except:
        break
browse_quit()
print('Scraping Complete')
wine_df.to_csv("chathambuyrite.csv")